In [1]:
import cv2
import numpy as np
import imgaug as ia
ia.seed(1)
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import os
import imageio
import re

In [2]:
annotations=pd.read_csv('./gray_train2/_annotations.csv')

In [3]:
print(annotations)

                                               filename  width  height  \
0     television_224_jpg.rf.eaa34149fd709165f94e4765...    416     416   
1     bed_10_jpg.rf.ea278f5f76d64997c5d276b475c5de35...    416     416   
2     wardrobe1115_jpg.rf.ec8f2958fa15e060ef14e38599...    416     416   
3     officechair_105_jpg.rf.eb134924538ad861b67cad2...    416     416   
4     wardrobe1248_jpg.rf.eb948d5b15f0c097ca8d171a1f...    416     416   
...                                                 ...    ...     ...   
1881  bed_16_jpg.rf.4741d8d9ac77b8f65570c3f43c117f40...    416     416   
1882  refrigerator_189_jpg.rf.474487f15431841185f1f9...    416     416   
1883  officechair_70_jpg.rf.3bfd85c51c4e7836d0db2ed2...    416     416   
1884  drawer_49_jpg.rf.432c0723810206664800a35613448...    416     416   
1885  wardrobe1543_jpg.rf.417ea7545d48aba2103ad62595...    416     416   

            class  xmin  ymin  xmax  ymax  
0     notwardrobe     7    12   405   397  
1     notwardrobe    52

In [4]:
image_augmentations = iaa.SomeOf( 1,
    [                                 
    # Scale the Images
    iaa.Affine(scale=(0.5, 1.5)),

    # Rotate the Images
    iaa.Affine(rotate=(-180, 180)),
        
    # Rotate the Images
    iaa.Affine(rotate=(-360, 360)),

    # Shift the Image
    iaa.Affine(translate_percent={"x":(-0.3, 0.3),"y":(-0.3, 0.3)}),

    # Flip the Image
    iaa.Fliplr(1),

    # Increase or decrease the brightness
    iaa.Multiply((0.7, 1.2)),

    # Add Gaussian Blur
    iaa.GaussianBlur(sigma=(1.0, 2.0)),
    
    # Add Gaussian Noise
    iaa.AdditiveGaussianNoise(scale=(0.03*255, 0.05*255))

])

In [5]:
def bounding_boxes_to_df(bounding_boxes_object):

    # Convert Bounding Boxes Object to Array
    bounding_boxes_array = bounding_boxes_object.to_xyxy_array()
    
    # Convert the array into DataFrame
    df_bounding_boxes = pd.DataFrame(bounding_boxes_array, 
                                     columns=['xmin', 'ymin', 'xmax', 'ymax'])
    
    # Return the DataFrame
    return df_bounding_boxes

In [6]:
def image_aug(df, images_path, aug_images_path, augmentor, multiple = 10):
    
    # Fill this DataFrame with image attributes
    augmentations_df = pd.DataFrame(
        columns=['filename','width','height','class', 'xmin', 'ymin', 'xmax', 'ymax'])
    
    # Group the data by filenames
    grouped_df = df.groupby('filename')

    # Create the directory for all augmentated images
    if not os.path.exists(aug_images_path):
        os.mkdir(aug_images_path)

    # Create directories for each class of augmentated images
    #for folder in df['class'].unique():
    # if not os.path.exists(os.path.join(aug_images_path, folder)):
    #   os.mkdir(os.path.join(aug_images_path, folder))

    for i in range(multiple):
        # Post Fix we add to the each different augmentation of one image
        image_postfix = str(i)
        
        # Loop to perform the augmentations
        
        for filename in df['filename'].unique():
            
            augname=filename[:-4]
            
            augmented_path = os.path.join(aug_images_path, filename[:-4])+image_postfix+'.jpg'

            # Take one image at a time with its information
            single_image = grouped_df.get_group(filename)
            single_image = single_image.reset_index()
            single_image = single_image.drop(['index'], axis=1)   
        
            # Read the image
            image = imageio.imread(os.path.join(images_path, filename))

            # Get bounding box
            bounding_box_array = single_image.drop(['filename', 'width', 'height',
                                                'class'], axis=1).values

            # Give the bounding box to imgaug library
            bounding_box = BoundingBoxesOnImage.from_xyxy_array(bounding_box_array, 
                                                            shape=image.shape)

            # Perform random 2 Augmentations
            image_aug, bounding_box_aug = augmentor(image=image, 
                                                bounding_boxes=bounding_box)
        
            # Discard the the bounding box going out the image completely   
            bounding_box_aug = bounding_box_aug.remove_out_of_image()

            # Clip the bounding box that are only partially out of th image
            bounding_box_aug = bounding_box_aug.clip_out_of_image()

            # Get rid of the the image if bounding box was discarded  
            if re.findall('Image...', str(bounding_box_aug)) == ['Image([]']:
                pass
        
            else:
            
                # Create the augmented image file
                imageio.imwrite(augmented_path, image_aug) 

                # Update the image width and height after augmentation
                info_df = single_image.drop(['xmin', 'ymin', 'xmax', 'ymax'], axis=1)    
                for index, _ in info_df.iterrows():
                    info_df.at[index, 'width'] = image_aug.shape[1]
                    info_df.at[index, 'height'] = image_aug.shape[0]
                
                # Add the prefix to each image to differentiate if required
                print(info_df['filename'])
                #info_df['filename'] = info_df['filename'].apply(lambda x: x + image_postfix + '.jpg')
                info_df['filename'] = info_df['filename'].apply(lambda x: x[:-4] + image_postfix + '.jpg')
                print(info_df['filename'])
            
                # Create the augmented bounding boxes dataframe 
                bounding_box_df = bounding_boxes_to_df(bounding_box_aug)

                # Concatenate the filenames, height, width and bounding boxes 
                aug_df = pd.concat([info_df, bounding_box_df], axis=1)

                # Add all the information to augmentations_df we initialized above
                augmentations_df = pd.concat([augmentations_df, aug_df])
            
        # Remove index
        augmentations_df = augmentations_df.reset_index()
        augmentations_df = augmentations_df.drop(['index'], axis=1)
            
    # Return the Dataframe
    return augmentations_df


In [7]:
augmented_images_df=image_aug(annotations, './gray_train2','./gray_train2/aug_gray_train_10',image_augmentations)

0    television_224_jpg.rf.eaa34149fd709165f94e4765...
Name: filename, dtype: object
0    television_224_jpg.rf.eaa34149fd709165f94e4765...
Name: filename, dtype: object
0    bed_10_jpg.rf.ea278f5f76d64997c5d276b475c5de35...
Name: filename, dtype: object
0    bed_10_jpg.rf.ea278f5f76d64997c5d276b475c5de35...
Name: filename, dtype: object
0    wardrobe1115_jpg.rf.ec8f2958fa15e060ef14e38599...
Name: filename, dtype: object
0    wardrobe1115_jpg.rf.ec8f2958fa15e060ef14e38599...
Name: filename, dtype: object
0    officechair_105_jpg.rf.eb134924538ad861b67cad2...
Name: filename, dtype: object
0    officechair_105_jpg.rf.eb134924538ad861b67cad2...
Name: filename, dtype: object
0    wardrobe1248_jpg.rf.eb948d5b15f0c097ca8d171a1f...
Name: filename, dtype: object
0    wardrobe1248_jpg.rf.eb948d5b15f0c097ca8d171a1f...
Name: filename, dtype: object
0    wardrobe34_jpg.rf.ea4e74e14851454457255d6d9721...
Name: filename, dtype: object
0    wardrobe34_jpg.rf.ea4e74e14851454457255d6d9721...
Name: file

Name: filename, dtype: object
0    bed_86_jpg.rf.f1df289e09b70e78a37998fa4e45aa26...
Name: filename, dtype: object
0    showcase_87_jpg.rf.f10f5390b8eafae9d0318958ea9...
Name: filename, dtype: object
0    showcase_87_jpg.rf.f10f5390b8eafae9d0318958ea9...
Name: filename, dtype: object
0    wardrobe1593_jpg.rf.f0db6ba594e82c9141c2a9d6f4...
Name: filename, dtype: object
0    wardrobe1593_jpg.rf.f0db6ba594e82c9141c2a9d6f4...
Name: filename, dtype: object
0    table_25_jpg.rf.f14929f94970816859b64d90d55d44...
Name: filename, dtype: object
0    table_25_jpg.rf.f14929f94970816859b64d90d55d44...
Name: filename, dtype: object
0    officechair_58_jpg.rf.f1144ce8afb99a4278097425...
Name: filename, dtype: object
0    officechair_58_jpg.rf.f1144ce8afb99a4278097425...
Name: filename, dtype: object
0    wardrobe1808_jpg.rf.f0c951b7650ba2aec412955541...
Name: filename, dtype: object
0    wardrobe1808_jpg.rf.f0c951b7650ba2aec412955541...
Name: filename, dtype: object
0    refrigerator_29_jpg.rf.eda63df

Name: filename, dtype: object
0    wardrobe1189_jpg.rf.f8eaf1228cb69a6421350c8d5c...
Name: filename, dtype: object
0    wardrobe1189_jpg.rf.f8eaf1228cb69a6421350c8d5c...
Name: filename, dtype: object
0    sofa_13_jpg.rf.f913560c4efb224824c29f2cc08dd0b...
Name: filename, dtype: object
0    sofa_13_jpg.rf.f913560c4efb224824c29f2cc08dd0b...
Name: filename, dtype: object
0    wardrobe337_jpg.rf.f90c13a9748a088ad3f945c4cc3...
Name: filename, dtype: object
0    wardrobe337_jpg.rf.f90c13a9748a088ad3f945c4cc3...
Name: filename, dtype: object
0    refrigerator_183_jpg.rf.f9d1f0d7d9e30afd2c0396...
Name: filename, dtype: object
0    refrigerator_183_jpg.rf.f9d1f0d7d9e30afd2c0396...
Name: filename, dtype: object
0    wardrobe1341_jpg.rf.f83382a8a28a6a7fc845fd7213...
Name: filename, dtype: object
0    wardrobe1341_jpg.rf.f83382a8a28a6a7fc845fd7213...
Name: filename, dtype: object
0    wardrobe1359_jpg.rf.f91f9fbe2778593c578a2f9082...
Name: filename, dtype: object
0    wardrobe1359_jpg.rf.f91f9fbe27

Name: filename, dtype: object
0    wardrobe1817_jpg.rf.ff6d700ec3cafd4a653f1d218f...
Name: filename, dtype: object
0    airconditioner_45_jpg.rf.fe9e0cac7c5d76759278c...
Name: filename, dtype: object
0    airconditioner_45_jpg.rf.fe9e0cac7c5d76759278c...
Name: filename, dtype: object
0    wardrobe1372_jpg.rf.fe86eec96a1ef0dbe7cae6d61c...
Name: filename, dtype: object
0    wardrobe1372_jpg.rf.fe86eec96a1ef0dbe7cae6d61c...
Name: filename, dtype: object
0    wardrobe356_jpg.rf.ff561e1b1fb16f656a519b0e140...
Name: filename, dtype: object
0    wardrobe356_jpg.rf.ff561e1b1fb16f656a519b0e140...
Name: filename, dtype: object
0    wardrobe390_jpg.rf.ff723b1d0d76c43e868360d3c06...
Name: filename, dtype: object
0    wardrobe390_jpg.rf.ff723b1d0d76c43e868360d3c06...
Name: filename, dtype: object
0    wardrobe1827_jpg.rf.67b2dbdc5820a5a2688a13d8a5...
Name: filename, dtype: object
0    wardrobe1827_jpg.rf.67b2dbdc5820a5a2688a13d8a5...
Name: filename, dtype: object
0    wardrobe1197_jpg.rf.67fce1bef9

0    wardrobe1389_jpg.rf.6e0630e9ca5d753f71c3253ffb...
Name: filename, dtype: object
0    wardrobe1389_jpg.rf.6e0630e9ca5d753f71c3253ffb...
Name: filename, dtype: object
0    wardrobe172_jpg.rf.6de2d35bb9a43966ad1c0188668...
Name: filename, dtype: object
0    wardrobe172_jpg.rf.6de2d35bb9a43966ad1c0188668...
Name: filename, dtype: object
0    wardrobe1416_jpg.rf.6e8103cf7358bb0b756a65b54d...
Name: filename, dtype: object
0    wardrobe1416_jpg.rf.6e8103cf7358bb0b756a65b54d...
Name: filename, dtype: object
0    lamp_168_jpg.rf.6bb55178c3ef6ee240ff1c7e768b0c...
Name: filename, dtype: object
0    lamp_168_jpg.rf.6bb55178c3ef6ee240ff1c7e768b0c...
Name: filename, dtype: object
0    desk_5_jpg.rf.6d8f780c8adb1eb38818d168597a59e9...
Name: filename, dtype: object
0    desk_5_jpg.rf.6d8f780c8adb1eb38818d168597a59e9...
Name: filename, dtype: object
0    wardrobe1161_jpg.rf.6f1b8874e63bf0665073335252...
Name: filename, dtype: object
0    wardrobe1161_jpg.rf.6f1b8874e63bf0665073335252...
Name: file

...

0    wardrobe1862_jpg.rf.287bc29c3b03f96243e6b222e6...
Name: filename, dtype: object
0    wardrobe1862_jpg.rf.287bc29c3b03f96243e6b222e6...
Name: filename, dtype: object
0    shelf_32_jpg.rf.28d5d53a68914ef133b57a5093b059...
Name: filename, dtype: object
0    shelf_32_jpg.rf.28d5d53a68914ef133b57a5093b059...
Name: filename, dtype: object
0    showcase_3_jpg.rf.283222e1f04a8190a86b19e8eb72...
Name: filename, dtype: object
0    showcase_3_jpg.rf.283222e1f04a8190a86b19e8eb72...
Name: filename, dtype: object
0    wardrobe1105_jpg.rf.27b8e93e628970da92adcffe6c...
Name: filename, dtype: object
0    wardrobe1105_jpg.rf.27b8e93e628970da92adcffe6c...
Name: filename, dtype: object
0    airconditioner_5_jpg.rf.28dde26ed36f3ae6aab7d1...
Name: filename, dtype: object
0    airconditioner_5_jpg.rf.28dde26ed36f3ae6aab7d1...
Name: filename, dtype: object
0    wardrobe1335_jpg.rf.28be9eeb409f4f9f9f25570771...
Name: filename, dtype: object
0    wardrobe1335_jpg.rf.28be9eeb409f4f9f9f25570771...
Name: file

0    desk_101_jpg.rf.2dca6aac7b11d6757c799f0e5dfda4...
Name: filename, dtype: object
0    desk_101_jpg.rf.2dca6aac7b11d6757c799f0e5dfda4...
Name: filename, dtype: object
0    drawer_40_jpg.rf.2e83207e0d3aff950d13f38f08f93...
Name: filename, dtype: object
0    drawer_40_jpg.rf.2e83207e0d3aff950d13f38f08f93...
Name: filename, dtype: object
0    wardrobe634_jpg.rf.30723f818b3ec8aa81a46b01270...
Name: filename, dtype: object
0    wardrobe634_jpg.rf.30723f818b3ec8aa81a46b01270...
Name: filename, dtype: object
0    officechair_85_jpg.rf.2e61c09150b5c4c4152ea860...
Name: filename, dtype: object
0    officechair_85_jpg.rf.2e61c09150b5c4c4152ea860...
Name: filename, dtype: object
0    showcase_111_jpg.rf.2d9624d4cc54d3b21ee3c18d39...
Name: filename, dtype: object
0    showcase_111_jpg.rf.2d9624d4cc54d3b21ee3c18d39...
Name: filename, dtype: object
0    desk_157_jpg.rf.2fca9510f9c8896825a86bc1947228...
Name: filename, dtype: object
0    desk_157_jpg.rf.2fca9510f9c8896825a86bc1947228...
Name: file

0    lamp_106_jpg.rf.375f6ba884f6df7c5fc4e7f2923c73...
Name: filename, dtype: object
0    lamp_106_jpg.rf.375f6ba884f6df7c5fc4e7f2923c73...
Name: filename, dtype: object
0    drawer_41_jpg.rf.356f217e37070faf6c9cf1bf09fb4...
Name: filename, dtype: object
0    drawer_41_jpg.rf.356f217e37070faf6c9cf1bf09fb4...
Name: filename, dtype: object
0    shelf_117_jpg.rf.37b7dfbb67500aef28046f36055d9...
Name: filename, dtype: object
0    shelf_117_jpg.rf.37b7dfbb67500aef28046f36055d9...
Name: filename, dtype: object
0    table_16_jpg.rf.3714c91ecfb534be1cdcedf48ef198...
Name: filename, dtype: object
0    table_16_jpg.rf.3714c91ecfb534be1cdcedf48ef198...
Name: filename, dtype: object
0    television_14_jpg.rf.356c70bb7ba64024b76546f55...
Name: filename, dtype: object
0    television_14_jpg.rf.356c70bb7ba64024b76546f55...
Name: filename, dtype: object
0    officechair_44_jpg.rf.3546df871feb81cfda2457b3...
Name: filename, dtype: object
0    officechair_44_jpg.rf.3546df871feb81cfda2457b3...
Name: file

0    refrigerator_173_jpg.rf.3b964a29014cfff8569168...
Name: filename, dtype: object
0    refrigerator_173_jpg.rf.3b964a29014cfff8569168...
Name: filename, dtype: object
0    wardrobe740_jpg.rf.3d304ee849dcb49c6d68e27ebae...
Name: filename, dtype: object
0    wardrobe740_jpg.rf.3d304ee849dcb49c6d68e27ebae...
Name: filename, dtype: object
0    wardrobe1465_jpg.rf.3c7cf67b8f81c7eeb3191a8e9c...
Name: filename, dtype: object
0    wardrobe1465_jpg.rf.3c7cf67b8f81c7eeb3191a8e9c...
Name: filename, dtype: object
0    showcase_101_jpg.rf.385dc31cb277e7e2aa11b840d9...
Name: filename, dtype: object
0    showcase_101_jpg.rf.385dc31cb277e7e2aa11b840d9...
Name: filename, dtype: object
0    officechair_26_jpg.rf.3c6a835cebe038b30d1e4e91...
Name: filename, dtype: object
0    officechair_26_jpg.rf.3c6a835cebe038b30d1e4e91...
Name: filename, dtype: object
0    wardrobe1311_jpg.rf.3bec87cc733ef979efae90c594...
Name: filename, dtype: object
0    wardrobe1311_jpg.rf.3bec87cc733ef979efae90c594...
Name: file

Name: filename, dtype: object
0    bed_33_jpg.rf.42e839fa0ab3f8c434712dfa516e6073...
Name: filename, dtype: object
0    wardrobe808_jpg.rf.43796402d2a8dfdccbf7a1848ef...
Name: filename, dtype: object
0    wardrobe808_jpg.rf.43796402d2a8dfdccbf7a1848ef...
Name: filename, dtype: object
0    lamp_31_jpg.rf.42ef2ff19469a42e53e34eb0a118f57...
Name: filename, dtype: object
0    lamp_31_jpg.rf.42ef2ff19469a42e53e34eb0a118f57...
Name: filename, dtype: object
0    wardrobe1683_jpg.rf.40e01515567b3aeb4d56f71d6c...
Name: filename, dtype: object
0    wardrobe1683_jpg.rf.40e01515567b3aeb4d56f71d6c...
Name: filename, dtype: object
0    wardrobe1162_jpg.rf.43887d0fa168d9c5d50324f031...
Name: filename, dtype: object
0    wardrobe1162_jpg.rf.43887d0fa168d9c5d50324f031...
Name: filename, dtype: object
0    television_13_jpg.rf.4418cb6789a1bb1174827ce0f...
Name: filename, dtype: object
0    television_13_jpg.rf.4418cb6789a1bb1174827ce0f...
Name: filename, dtype: object
0    wardrobe1019_jpg.rf.447acde11c

In [8]:
print(augmented_images_df)

                                                filename width height  \
0      television_224_jpg.rf.eaa34149fd709165f94e4765...   416    416   
1      bed_10_jpg.rf.ea278f5f76d64997c5d276b475c5de35...   416    416   
2      wardrobe1115_jpg.rf.ec8f2958fa15e060ef14e38599...   416    416   
3      officechair_105_jpg.rf.eb134924538ad861b67cad2...   416    416   
4      wardrobe1248_jpg.rf.eb948d5b15f0c097ca8d171a1f...   416    416   
...                                                  ...   ...    ...   
18850  bed_16_jpg.rf.4741d8d9ac77b8f65570c3f43c117f40...   416    416   
18851  refrigerator_189_jpg.rf.474487f15431841185f1f9...   416    416   
18852  officechair_70_jpg.rf.3bfd85c51c4e7836d0db2ed2...   416    416   
18853  drawer_49_jpg.rf.432c0723810206664800a35613448...   416    416   
18854  wardrobe1543_jpg.rf.417ea7545d48aba2103ad62595...   416    416   

             class        xmin        ymin        xmax        ymax  
0      notwardrobe    7.000000   12.000000  405.000000

In [9]:
augmented_images_df=augmented_images_df.sort_values('filename',ignore_index=True)
augmented_images_df.to_csv('aug_gray_train_10.csv')